In [15]:
import sqlite3
from fastapi import FastAPI, HTTPException
from fastapi.responses import JSONResponse
from pydantic import BaseModel
import uvicorn
from threading import Thread


In [20]:
class MenuItem(BaseModel):
    id: int = None
    name: str
    description: str
    price: float

In [18]:
def init_db():
    with sqlite3.connect('menu.db') as conn:
        curr = conn.cursor()
        curr.execute('''
            CREATE TABLE IF NOT EXISTS menu_items (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                name TEXT NOT NULL,
                description TEXT,
                price REAL NOT NULL
            )
        ''')

In [27]:
def get_menu_items() -> list[MenuItem]:
    with sqlite3.connect("menu.db") as conn:
        cur = conn.cursor()
        try:
            results = cur.execute("""select * from menu_items""")
            return results.fetchall()
        except Exception as e:
            print(f"Exception error {e}")
    #return [MenuItem(id=row[0], name=row[1], description=row[2], price=row[3]) for row in rows]

In [23]:
def insert_menu_item(item: MenuItem) -> list[tuple]:
    with sqlite3.connect('menu.db') as conn:
        curr = conn.cursor()
        curr.execute('''
            INSERT INTO menu_items (name, description, price) VALUES (?, ?, ?)
        ''', (item.name, item.description, item.price))
    return get_menu_items()

In [28]:
ss = MenuItem(name= "salad" , description= "tomato with cucomber", price=10)
insert_menu_item(ss)

[(1, 'salad', 'tomato with cucomber', 10.0),
 (2, 'salad', 'tomato with cucomber', 10.0)]

In [30]:
sm = MenuItem(name= "burger" , description= "2 pices of angos and souce", price=15)
insert_menu_item(sm)

[(1, 'salad', 'tomato with cucomber', 10.0),
 (2, 'salad', 'tomato with cucomber with wheezing', 12.0),
 (3, 'burger', '2 pices of angos and souce', 15.0)]

In [24]:
def update_menu_item(item: MenuItem) -> list[tuple]:
    with sqlite3.connect('menu.db') as conn:
        curr = conn.cursor()
        curr.execute('''
            UPDATE menu_items SET name=?, description=?, price=? WHERE id=?
        ''', (item.name, item.description, item.price, item.id))
    return get_menu_items()

In [29]:
upd = MenuItem(name= "salad" , description= "tomato with cucomber with wheezing", price=12, id = 2)
update_menu_item(upd)

[(1, 'salad', 'tomato with cucomber', 10.0),
 (2, 'salad', 'tomato with cucomber with wheezing', 12.0)]

In [25]:
def delete_menu_item(item_id: int) -> list[tuple]:
    with sqlite3.connect('menu.db') as conn:
        curr = conn.cursor()
        curr.execute('DELETE FROM menu_items WHERE id=?', (item_id,))
    return get_menu_items()

In [31]:
delete_menu_item(2)

[(1, 'salad', 'tomato with cucomber', 10.0),
 (3, 'burger', '2 pices of angos and souce', 15.0)]

In [ ]:
init_db()

In [32]:
app = FastAPI()

In [33]:
@app.get("/menu/")
async def read_menu():
    return get_menu_items()

In [34]:
@app.post("/menu/")
async def create_menu_item(item: MenuItem):
    return insert_menu_item(item)

In [35]:
@app.put("/menu/")
async def update_item(item: MenuItem):
    return update_menu_item(item)

In [36]:
@app.delete("/menu/{item_id}")
async def delete_item(item_id: int):
    return delete_menu_item(item_id)

In [37]:
def run_uvicorn():
    uvicorn.run(app, host="127.0.0.1", port=8000)

In [38]:
# Run Uvicorn in a separate thread
if __name__ == "__main__":
    server_thread = Thread(target=run_uvicorn)
    server_thread.start()

INFO:     Started server process [2112]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:56124 - "GET / HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:56124 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:56125 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:56125 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:56198 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:56201 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:56205 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:56207 - "GET / HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:56228 - "GET / HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:56228 - "GET / HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:56240 - "GET / HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:56258 - "GET /menu/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:56394 - "GET /menu/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:56405 - "POST /menu/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:56426 - "POST /menu/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:56447 - "PUT /menu/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:56475 - "PUT /menu/ HTTP/1.1" 200 OK
INF